## Step1: Install third-party libraries

In [1]:
!pip3 install -U pip
!pip3 install -U setuptools wheel
# Install the proper version of PyTorch following https://pytorch.org/get-started/locally/
!pip3 install torch==1.12.0+cu113 torchvision==0.13.0+cu113 torchtext==0.13.0 --extra-index-url https://download.pytorch.org/whl/cu113
!pip3 install autogluon==0.6.0
!pip3 install Pillow==9.0.0
!pip3 install vowpalwabbit==8.10.1

Mounted at /content/drive
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 21.3 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 22.0.4
    Uninstalling pip-22.0.4:
      Successfully uninstalled pip-22.0.4
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Found existing installation: lightgbm 3.3.5
Uninstalling lightgbm-3.3.5:
  Successfully uninstalled lightgbm-3.3.5
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/, https://download.pytorch.org/whl/cu113
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 GB 896.7 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.4/23.4 MB 56.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 23.1 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installatio

'\n重新启动代码执行程序！\n'

# Step2: Import libraries

In [1]:
%matplotlib inline
import numpy as np
import warnings
import matplotlib.pyplot as plt
import pandas as pd
warnings.filterwarnings('ignore')
np.random.seed(123)
from autogluon.tabular import TabularDataset, TabularPredictor
from autogluon.common.utils.utils import setup_outputdir
from autogluon.core.utils.loaders import load_pkl
from autogluon.core.utils.savers import save_pkl
import os.path

# Step3: Define necessary functions

In [ ]:
class MultilabelPredictor():
    """ Tabular Predictor for predicting multiple columns in table.
        Creates multiple TabularPredictor objects which you can also use individually.
        You can access the TabularPredictor for a particular label via: `multilabel_predictor.get_predictor(label_i)`

        Parameters
        ----------
        labels : List[str]
            The ith element of this list is the column (i.e. `label`) predicted by the ith TabularPredictor stored in this object.
        path : str
            Path to directory where models and intermediate outputs should be saved.
            If unspecified, a time-stamped folder called "AutogluonModels/ag-[TIMESTAMP]" will be created in the working directory to store all models.
            Note: To call `fit()` twice and save all results of each fit, you must specify different `path` locations or don't specify `path` at all.
            Otherwise files from first `fit()` will be overwritten by second `fit()`.
            Caution: when predicting many labels, this directory may grow large as it needs to store many TabularPredictors.
        problem_types : List[str]
            The ith element is the `problem_type` for the ith TabularPredictor stored in this object.
        eval_metrics : List[str]
            The ith element is the `eval_metric` for the ith TabularPredictor stored in this object.
        consider_labels_correlation : bool
            Whether the predictions of multiple labels should account for label correlations or predict each label independently of the others.
            If True, the ordering of `labels` may affect resulting accuracy as each label is predicted conditional on the previous labels appearing earlier in this list (i.e. in an auto-regressive fashion).
            Set to False if during inference you may want to individually use just the ith TabularPredictor without predicting all the other labels.
        kwargs :
            Arguments passed into the initialization of each TabularPredictor.

    """

    multi_predictor_file = 'multilabel_predictor.pkl'

    def __init__(self, labels, path, problem_types=None, eval_metrics=None, consider_labels_correlation=True, **kwargs):
        if len(labels) < 2:
            raise ValueError("MultilabelPredictor is only intended for predicting MULTIPLE labels (columns), use TabularPredictor for predicting one label (column).")
        self.path = setup_outputdir(path, warn_if_exist=False)
        self.labels = labels
        self.consider_labels_correlation = consider_labels_correlation
        self.predictors = {}  # key = label, value = TabularPredictor or str path to the TabularPredictor for this label
        if eval_metrics is None:
            self.eval_metrics = {}
        else:
            self.eval_metrics = {labels[i] : eval_metrics[i] for i in range(len(labels))}
        problem_type = None
        eval_metric = None
        for i in range(len(labels)):
            label = labels[i]
            path_i = self.path + "Predictor_" + label
            if problem_types is not None:
                problem_type = problem_types[i]
            if eval_metrics is not None:
                eval_metric = self.eval_metrics[label]
            self.predictors[label] = TabularPredictor(label=label, problem_type=problem_type, eval_metric=eval_metric, path=path_i, **kwargs)

    def fit(self, train_data, tuning_data=None, **kwargs):
        """ Fits a separate TabularPredictor to predict each of the labels.

            Parameters
            ----------
            train_data, tuning_data : str or autogluon.tabular.TabularDataset or pd.DataFrame
                See documentation for `TabularPredictor.fit()`.
            kwargs :
                Arguments passed into the `fit()` call for each TabularPredictor.
        """
        if isinstance(train_data, str):
            train_data = TabularDataset(train_data)
        if tuning_data is not None and isinstance(tuning_data, str):
            tuning_data = TabularDataset(tuning_data)
        train_data_og = train_data.copy()
        if tuning_data is not None:
            tuning_data_og = tuning_data.copy()
        else:
            tuning_data_og = None
        save_metrics = len(self.eval_metrics) == 0
        for i in range(len(self.labels)):
            label = self.labels[i]
            predictor = self.get_predictor(label)
            if not self.consider_labels_correlation:
                labels_to_drop = [l for l in self.labels if l != label]
            else:
                labels_to_drop = [self.labels[j] for j in range(i+1, len(self.labels))]
            train_data = train_data_og.drop(labels_to_drop, axis=1)
            if tuning_data is not None:
                tuning_data = tuning_data_og.drop(labels_to_drop, axis=1)
            print(f"Fitting TabularPredictor for label: {label} ...")
            predictor.fit(train_data=train_data, tuning_data=tuning_data, **kwargs)
            self.predictors[label] = predictor.path
            if save_metrics:
                self.eval_metrics[label] = predictor.eval_metric
            copy_trained(label)
        self.save()

    def predict(self, data, **kwargs):
        """ Returns DataFrame with label columns containing predictions for each label.

            Parameters
            ----------
            data : str or autogluon.tabular.TabularDataset or pd.DataFrame
                Data to make predictions for. If label columns are present in this data, they will be ignored. See documentation for `TabularPredictor.predict()`.
            kwargs :
                Arguments passed into the predict() call for each TabularPredictor.
        """
        return self._predict(data, as_proba=False, **kwargs)

    def predict_proba(self, data, **kwargs):
        """ Returns dict where each key is a label and the corresponding value is the `predict_proba()` output for just that label.

            Parameters
            ----------
            data : str or autogluon.tabular.TabularDataset or pd.DataFrame
                Data to make predictions for. See documentation for `TabularPredictor.predict()` and `TabularPredictor.predict_proba()`.
            kwargs :
                Arguments passed into the `predict_proba()` call for each TabularPredictor (also passed into a `predict()` call).
        """
        return self._predict(data, as_proba=True, **kwargs)

    def evaluate(self, data, **kwargs):
        """ Returns dict where each key is a label and the corresponding value is the `evaluate()` output for just that label.

            Parameters
            ----------
            data : str or autogluon.tabular.TabularDataset or pd.DataFrame
                Data to evalate predictions of all labels for, must contain all labels as columns. See documentation for `TabularPredictor.evaluate()`.
            kwargs :
                Arguments passed into the `evaluate()` call for each TabularPredictor (also passed into the `predict()` call).
        """
        data = self._get_data(data)
        eval_dict = {}
        for label in self.labels:
            print(f"Evaluating TabularPredictor for label: {label} ...")
            predictor = self.get_predictor(label)
            eval_dict[label] = predictor.evaluate(data, **kwargs)
            if self.consider_labels_correlation:
                data[label] = predictor.predict(data, **kwargs)
        return eval_dict

    def save(self):
        """ Save MultilabelPredictor to disk. """
        for label in self.labels:
            if not isinstance(self.predictors[label], str):
                self.predictors[label] = self.predictors[label].path
        save_pkl.save(path=self.path+self.multi_predictor_file, object=self)
        try:
          save_pkl.save(path="/content/drive/MyDrive/OSDG16/model/"+self.multi_predictor_file, object=self)
        except:
          print("failed when save to /content/drive/MyDrive/OSDG16/model file.")
          print("self.path：",self.path)
        print(f"MultilabelPredictor saved to disk. Load with: MultilabelPredictor.load('{self.path}')")

    @classmethod
    def load(cls, path):
        """ Load MultilabelPredictor from disk `path` previously specified when creating this MultilabelPredictor. """
        path = os.path.expanduser(path)
        if path[-1] != os.path.sep:
            path = path + os.path.sep
        return load_pkl.load(path=path+cls.multi_predictor_file)

    def get_predictor(self, label):
        """ Returns TabularPredictor which is used to predict this label. """
        predictor = self.predictors[label]
        if isinstance(predictor, str):
            return TabularPredictor.load(path=predictor)
        return predictor

    def _get_data(self, data):
        if isinstance(data, str):
            return TabularDataset(data)
        return data.copy()

    def _predict(self, data, as_proba=False, **kwargs):
        data = self._get_data(data)
        if as_proba:
            predproba_dict = {}
        for label in self.labels:
            print(f"Predicting with TabularPredictor for label: {label} ...")
            predictor = self.get_predictor(label)
            if as_proba:
                predproba_dict[label] = predictor.predict_proba(data, as_multiclass=True, **kwargs)
            data[label] = predictor.predict(data, **kwargs)
        if not as_proba:
            return data[self.labels]
        else:
            return predproba_dict

# Step4: Import the testing dataset

In [2]:
import pandas as pd
test=pd.read_excel("/content/drive/MyDrive/OSDG16/english_library_test.xlsx")
test["text"]=test["title"]+' '+test['descriptions']
test=test.drop(columns=["title",'descriptions'])
test

,text
0,Do No Harm: The Rise of New Humanitarianism H...
1,Money vs. Barter History and Concepts This mod...
2,21st Century Perspectives on Macroeconomics In...
3,A Global Mitigation Scenario The three pillars...
4,A Job for the Future: The Innovation Broker Th...
...,...
895,Why Invest in Transboundary Basin Activities? ...
896,Winners and losers in the global economy In th...
897,Work in a Post Manufacturing World This video ...
898,Youth and Food: New Entrepreneurs in the Food ...


# Step5: Predict using trained predictors

In [3]:
predictions=pd.DataFrame(columns=["sdg_1","sdg_2","sdg_3","sdg_4","sdg_5","sdg_6","sdg_7","sdg_8","sdg_9","sdg_10","sdg_11","sdg_12","sdg_13","sdg_14","sdg_15","sdg_16"])
for i in range(1,17):
    predictor = TabularPredictor.load("/content/drive/MyDrive/OSDG16/V2model/Predictor_sdg_"+str(i),require_py_version_match=False)
    res=predictor.predict_proba(test)
    predictions["sdg_"+str(i)]=res[1]
    print(str(i)+" success!")
    predictions.to_excel("prediction.xlsx")
predictions["text"]=test["text"]
predictions.to_excel("prediction.xlsx")

Found 1 mismatches between original and current metadata:
INFO:automm:Load pretrained checkpoint: /content/drive/MyDrive/OSDG16/V2model/Predictor_sdg_1/models/TextPredictor_BAG_L1/S1F1/text_nn/model.ckpt


Predicting: 0it [00:00, ?it/s]

INFO:automm:Load pretrained checkpoint: /content/drive/MyDrive/OSDG16/V2model/Predictor_sdg_1/models/TextPredictor_BAG_L1/S1F2/text_nn/model.ckpt


Predicting: 0it [00:00, ?it/s]

INFO:automm:Load pretrained checkpoint: /content/drive/MyDrive/OSDG16/V2model/Predictor_sdg_1/models/TextPredictor_BAG_L1/S1F3/text_nn/model.ckpt


...

INFO:automm:Load pretrained checkpoint: /content/drive/MyDrive/OSDG16/V2model/Predictor_sdg_16/models/TextPredictor_BAG_L1/S1F7/text_nn/model.ckpt


Predicting: 0it [00:00, ?it/s]

INFO:automm:Load pretrained checkpoint: /content/drive/MyDrive/OSDG16/V2model/Predictor_sdg_16/models/TextPredictor_BAG_L1/S1F8/text_nn/model.ckpt


Predicting: 0it [00:00, ?it/s]

16 success!


In [4]:
from copy import deepcopy
import pandas as pd
def convert(df,thres):
    newdf=deepcopy(df)
    newdf=newdf.drop(columns=["text"])
    for i in (newdf.index):
    for j in (newdf.columns):
        if newdf.loc[i,j]>=thres:
        newdf.loc[i,j]=1
        else:
        newdf.loc[i,j]=0
    return newdf
def save(df):
    df_0_5=convert(df,0.5)
    df_0_5["text"]=df["text"]
    df_0_7=convert(df,0.7)
    df_0_7["text"]=df["text"]
    with pd.ExcelWriter("prediction_with_threshold.xlsx") as writer:
        df.to_excel(writer,sheet_name="prob_output")
        df_0_5.to_excel(writer,sheet_name="thres=0.5")
        df_0_7.to_excel(writer,sheet_name="thres=0.7")

prediction=pd.read_excel("prediction.xlsx")
save(prediction)

# Result display

In [5]:
import pandas as pd
prediction=pd.read_excel("prediction.xlsx").drop(columns=["Unnamed: 0"])
prediction

,sdg_1,sdg_2,sdg_3,sdg_4,sdg_5,sdg_6,sdg_7,sdg_8,sdg_9,sdg_10,sdg_11,sdg_12,sdg_13,sdg_14,sdg_15,sdg_16,text
0,0.049449,0.000532,0.132726,0.015150,0.037724,0.000431,0.000466,0.001819,0.000005,0.010628,0.000987,0.085942,0.067440,0.000062,0.144495,0.870191,Do No Harm: The Rise of New Humanitarianism H...
1,0.054459,0.002047,0.131584,0.015132,0.037640,0.000405,0.000363,0.393613,0.000042,0.019476,0.000629,0.086737,0.066187,0.000058,0.144557,0.545428,Money vs. Barter History and Concepts This mod...
2,0.048197,0.000034,0.131552,0.015134,0.037652,0.000095,0.000321,0.017304,0.000014,0.009701,0.000364,0.085827,0.066418,0.000065,0.144404,0.968936,21st Century Perspectives on Macroeconomics In...
3,0.048269,0.000021,0.131566,0.015132,0.037718,0.000074,0.987147,0.000101,0.000010,0.008684,0.000097,0.086502,0.085362,0.000134,0.146069,0.000706,A Global Mitigation Scenario The three pillars...
4,0.048585,0.000512,0.131722,0.015134,0.037644,0.011135,0.003992,0.196602,0.000388,0.008795,0.002622,0.405267,0.067660,0.000155,0.144459,0.016509,A Job for the Future: The Innovation Broker Th...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
895,0.049226,0.002597,0.132061,0.015138,0.038010,0.922473,0.001912,0.000190,0.000009,0.009464,0.001837,0.085984,0.066831,0.000075,0.144563,0.002491,Why Invest in Transboundary Basin Activities? ...
896,0.119709,0.002285,0.131774,0.016515,0.038633,0.000353,0.001615,0.131725,0.000813,0.037032,0.000868,0.398663,0.066871,0.000106,0.145375,0.001018,Winners and losers in the global economy In th...
897,0.059287,0.004324,0.131890,0.015317,0.038149,0.000593,0.005373,0.437808,0.526703,0.012551,0.001662,0.127192,0.068831,0.000136,0.145811,0.016752,Work in a Post Manufacturing World This video ...
898,0.048402,0.491815,0.131550,0.018193,0.037635,0.000103,0.003835,0.003867,0.000207,0.008507,0.000185,0.088084,0.066184,0.000065,0.144386,0.077125,Youth and Food: New Entrepreneurs in the Food ...


In [6]:
prediction_0_7_threshold=pd.read_excel("prediction_with_threshold.xlsx",sheet_name="thres=0.7").drop(columns=["Unnamed: 0","Unnamed: 0.1"])
prediction_0_7_threshold

,sdg_1,sdg_2,sdg_3,sdg_4,sdg_5,sdg_6,sdg_7,sdg_8,sdg_9,sdg_10,sdg_11,sdg_12,sdg_13,sdg_14,sdg_15,sdg_16,text
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,Do No Harm: The Rise of New Humanitarianism H...
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Money vs. Barter History and Concepts This mod...
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,21st Century Perspectives on Macroeconomics In...
3,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,A Global Mitigation Scenario The three pillars...
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,A Job for the Future: The Innovation Broker Th...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
895,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,Why Invest in Transboundary Basin Activities? ...
896,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Winners and losers in the global economy In th...
897,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Work in a Post Manufacturing World This video ...
898,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Youth and Food: New Entrepreneurs in the Food ...
